In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import sklearn

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline



import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/My Drive/soccer_data.csv')
data

,Unnamed: 0,Unnamed: 0.1,statsbomb_xg,end_location,freeze_frame,player_position,first_time,aerial_won,deflected,open_goal,saved_off_target,saved_to_post,redirect,follows_dribble,kick_off,shot_outcome,shot_technique,shot_type,shot_body,x_pos,y_pos,Corner,Free Kick,Kick Off,Open Play,Penalty,Backheel,Diving Header,Half Volley,Lob,Normal,Overhead Kick,Volley,Blocked,Goal,Off T,Post,Saved,Saved Off Target,Saved to Post,Wayward,distance
0,0,0,0.021567,"[116.9, 41.6, 2.2]","[{'location': [89.6, 38.9], 'player': {'id': 6...","[99.2, 55.9]",0,0,0,0,0,0,0,0,0,Saved,Normal,Open Play,Right Foot,99.2,55.9,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,26.120490
1,1,1,0.018274,"[120.0, 44.7, 3.1]","[{'location': [117.2, 38.6], 'player': {'id': ...","[94.5, 25.1]",0,0,0,0,0,0,0,0,0,Off T,Normal,Open Play,Left Foot,94.5,25.1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,29.584624
2,2,2,0.016565,"[99.1, 31.3]","[{'location': [102.7, 32.4], 'player': {'id': ...","[95.3, 27.2]",0,0,0,0,0,0,0,0,0,Wayward,Normal,Open Play,Right Foot,95.3,27.2,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,27.865750
3,3,3,0.036084,"[120.0, 36.9, 0.3]","[{'location': [108.8, 43.1], 'player': {'id': ...","[106.0, 33.1]",1,0,0,0,0,0,0,0,0,Goal,Half Volley,Open Play,Right Foot,106.0,33.1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,15.652476
4,4,4,0.060245,"[120.0, 35.8, 3.8]","[{'location': [109.7, 36.0], 'player': {'id': ...","[108.1, 66.8]",1,0,0,0,0,0,0,0,0,Off T,Normal,Open Play,Left Foot,108.1,66.8,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,29.231832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22327,22679,22679,0.091634,"[120.0, 39.3, 1.2]","[{'location': [103.9, 61.2], 'player': {'id': ...","[93.6, 75.6]",0,0,0,0,0,0,0,0,0,Goal,Normal,Free Kick,Left Foot,93.6,75.6,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,44.240366
22328,22680,22680,0.060538,"[120.0, 22.8, 0.8]","[{'location': [97.6, 45.2], 'player': {'id': 3...","[108.0, 41.7]",0,0,0,0,0,0,0,0,0,Off T,Normal,Open Play,Head,108.0,41.7,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,12.106197
22329,22681,22681,0.154427,"[120.0, 43.4, 0.8]","[{'location': [108.0, 34.1], 'player': {'id': ...","[101.2, 40.0]",0,0,0,0,0,0,0,0,0,Goal,Normal,Free Kick,Left Foot,101.2,40.0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,18.800266
22330,22682,22682,0.273795,"[118.8, 41.1, 2.0]","[{'location': [104.2, 30.8], 'player': {'id': ...","[114.8, 41.2]",0,1,0,0,0,0,0,0,0,Saved,Normal,Open Play,Head,114.8,41.2,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,5.315073


#Cleaning Freeze Frame

Extracting teammate and opponent columns from a column named "Freeze Frame"

In [ ]:

tech = data['freeze_frame'].tolist()
listholder = []
#import string
for po in tech:
    try:
        d = eval(po)
        #print('success')
        listholder.append(d)
    except (TypeError, NameError):
        d = []
        listholder.append(d)



In [ ]:
try1 = listholder[0]
false_d = []
#for i in listholder:
#    for eachplayer in i:
#df = pd.DataFrame.from_dict(try1, orient ='index')
each_freeframe = []

for i in listholder:
    locationt = []
    playert = []
    positiont = []
    locationp = []
    playerp = []
    positionp = []
    for a in i:
        try:
            location_i = a.get('location')
            position_i = a.get('position')
            player_i = a.get('player')
            if a.get('teammate'):
                locationt.append(location_i)
                positiont.append(position_i)
                playert.append(player_i)
            else:
                locationp.append(location_i)
                positionp.append(position_i)
                playerp.append(player_i)
        except (TypeError, AttributeError):
            d = {}
            false_d.append(d)
    freezeframe_t = pd.DataFrame({'position': positiont,'location':locationt,'player':playert})
    freezeframe_p = pd.DataFrame({'position': positionp,'location':locationp,'player':playerp})
    combine = [freezeframe_t,freezeframe_p]
    each_freeframe = each_freeframe + [combine]
        
        
        
        

#Nearby Players

Here in each element in the each freeze frame holds a dataframe. The first dataframe hold all the information of the players on the field that's from the same team. The second dataframe hold all the information of the players on the fielld that's from the opponent team. 

In [ ]:
teammate_info = [i[0] for i in each_freeframe]
oppnent_info = [i[1] for i in each_freeframe]
temp = data.copy()

In [ ]:
data['teammate_info'] = teammate_info
data['oppnent_info'] = oppnent_info
data


,Unnamed: 0,Unnamed: 0.1,statsbomb_xg,end_location,freeze_frame,player_position,first_time,aerial_won,deflected,open_goal,saved_off_target,saved_to_post,redirect,follows_dribble,kick_off,shot_outcome,shot_technique,shot_type,shot_body,x_pos,y_pos,Corner,Free Kick,Kick Off,Open Play,Penalty,Backheel,Diving Header,Half Volley,Lob,Normal,Overhead Kick,Volley,Blocked,Goal,Off T,Post,Saved,Saved Off Target,Saved to Post,Wayward,distance,teammate_info,oppnent_info
0,0,0,0.021567,"[116.9, 41.6, 2.2]","[{'location': [89.6, 38.9], 'player': {'id': 6...","[99.2, 55.9]",0,0,0,0,0,0,0,0,0,Saved,Normal,Open Play,Right Foot,99.2,55.9,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,26.120490,position...,posi...
1,1,1,0.018274,"[120.0, 44.7, 3.1]","[{'location': [117.2, 38.6], 'player': {'id': ...","[94.5, 25.1]",0,0,0,0,0,0,0,0,0,Off T,Normal,Open Play,Left Foot,94.5,25.1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,29.584624,position...,pos...
2,2,2,0.016565,"[99.1, 31.3]","[{'location': [102.7, 32.4], 'player': {'id': ...","[95.3, 27.2]",0,0,0,0,0,0,0,0,0,Wayward,Normal,Open Play,Right Foot,95.3,27.2,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,27.865750,position...,pos...
3,3,3,0.036084,"[120.0, 36.9, 0.3]","[{'location': [108.8, 43.1], 'player': {'id': ...","[106.0, 33.1]",1,0,0,0,0,0,0,0,0,Goal,Half Volley,Open Play,Right Foot,106.0,33.1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,15.652476,position...,posi...
4,4,4,0.060245,"[120.0, 35.8, 3.8]","[{'location': [109.7, 36.0], 'player': {'id': ...","[108.1, 66.8]",1,0,0,0,0,0,0,0,0,Off T,Normal,Open Play,Left Foot,108.1,66.8,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,29.231832,posi...,posi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22327,22679,22679,0.091634,"[120.0, 39.3, 1.2]","[{'location': [103.9, 61.2], 'player': {'id': ...","[93.6, 75.6]",0,0,0,0,0,0,0,0,0,Goal,Normal,Free Kick,Left Foot,93.6,75.6,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,44.240366,posi...,posi...
22328,22680,22680,0.060538,"[120.0, 22.8, 0.8]","[{'location': [97.6, 45.2], 'player': {'id': 3...","[108.0, 41.7]",0,0,0,0,0,0,0,0,0,Off T,Normal,Open Play,Head,108.0,41.7,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,12.106197,position ...,position ...
22329,22681,22681,0.154427,"[120.0, 43.4, 0.8]","[{'location': [108.0, 34.1], 'player': {'id': ...","[101.2, 40.0]",0,0,0,0,0,0,0,0,0,Goal,Normal,Free Kick,Left Foot,101.2,40.0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,18.800266,posi...,pos...
22330,22682,22682,0.273795,"[118.8, 41.1, 2.0]","[{'location': [104.2, 30.8], 'player': {'id': ...","[114.8, 41.2]",0,1,0,0,0,0,0,0,0,Saved,Normal,Open Play,Head,114.8,41.2,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,5.315073,posi...,posi...


In [ ]:
data["oppnent_info"].iloc[0]["location"]

0     [89.6, 38.9]
1     [99.0, 57.7]
2     [98.9, 55.2]
3    [100.8, 48.9]
4    [104.7, 52.2]
5     [93.2, 30.0]
6    [105.8, 40.6]
7    [117.0, 42.3]
Name: location, dtype: object

In [ ]:
def num_players(pl, ol, d):
  nearby_players = 0
  for i in ol:
    if(i[0] > pl[0] - 1):
      distance = np.sqrt((pl[0] - i[0])**2 + (pl[1] - i[1])**2)
      if(distance <= d):
        nearby_players += 1
  return nearby_players


In [ ]:
nearby_col =[]
for i in range(data.shape[0]):
  nearby_col += [num_players([data["x_pos"].iloc[i],data["y_pos"].iloc[i]],data["oppnent_info"].iloc[i]["location"], 1.8)]


In [ ]:
data["nearby_players"] = nearby_col
data

,Unnamed: 0,Unnamed: 0.1,statsbomb_xg,end_location,freeze_frame,player_position,first_time,aerial_won,deflected,open_goal,saved_off_target,saved_to_post,redirect,follows_dribble,kick_off,shot_outcome,shot_technique,shot_type,shot_body,x_pos,y_pos,Corner,Free Kick,Kick Off,Open Play,Penalty,Backheel,Diving Header,Half Volley,Lob,Normal,Overhead Kick,Volley,Blocked,Goal,Off T,Post,Saved,Saved Off Target,Saved to Post,Wayward,distance,teammate_info,oppnent_info,nearby_players
0,0,0,0.021567,"[116.9, 41.6, 2.2]","[{'location': [89.6, 38.9], 'player': {'id': 6...","[99.2, 55.9]",0,0,0,0,0,0,0,0,0,Saved,Normal,Open Play,Right Foot,99.2,55.9,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,26.120490,position...,posi...,1
1,1,1,0.018274,"[120.0, 44.7, 3.1]","[{'location': [117.2, 38.6], 'player': {'id': ...","[94.5, 25.1]",0,0,0,0,0,0,0,0,0,Off T,Normal,Open Play,Left Foot,94.5,25.1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,29.584624,position...,pos...,1
2,2,2,0.016565,"[99.1, 31.3]","[{'location': [102.7, 32.4], 'player': {'id': ...","[95.3, 27.2]",0,0,0,0,0,0,0,0,0,Wayward,Normal,Open Play,Right Foot,95.3,27.2,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,27.865750,position...,pos...,0
3,3,3,0.036084,"[120.0, 36.9, 0.3]","[{'location': [108.8, 43.1], 'player': {'id': ...","[106.0, 33.1]",1,0,0,0,0,0,0,0,0,Goal,Half Volley,Open Play,Right Foot,106.0,33.1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,15.652476,position...,posi...,0
4,4,4,0.060245,"[120.0, 35.8, 3.8]","[{'location': [109.7, 36.0], 'player': {'id': ...","[108.1, 66.8]",1,0,0,0,0,0,0,0,0,Off T,Normal,Open Play,Left Foot,108.1,66.8,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,29.231832,posi...,posi...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22327,22679,22679,0.091634,"[120.0, 39.3, 1.2]","[{'location': [103.9, 61.2], 'player': {'id': ...","[93.6, 75.6]",0,0,0,0,0,0,0,0,0,Goal,Normal,Free Kick,Left Foot,93.6,75.6,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,44.240366,posi...,posi...,0
22328,22680,22680,0.060538,"[120.0, 22.8, 0.8]","[{'location': [97.6, 45.2], 'player': {'id': 3...","[108.0, 41.7]",0,0,0,0,0,0,0,0,0,Off T,Normal,Open Play,Head,108.0,41.7,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,12.106197,position ...,position ...,0
22329,22681,22681,0.154427,"[120.0, 43.4, 0.8]","[{'location': [108.0, 34.1], 'player': {'id': ...","[101.2, 40.0]",0,0,0,0,0,0,0,0,0,Goal,Normal,Free Kick,Left Foot,101.2,40.0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,18.800266,posi...,pos...,0
22330,22682,22682,0.273795,"[118.8, 41.1, 2.0]","[{'location': [104.2, 30.8], 'player': {'id': ...","[114.8, 41.2]",0,1,0,0,0,0,0,0,0,Saved,Normal,Open Play,Head,114.8,41.2,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,5.315073,posi...,posi...,2


#Angle Feature

Using Law of Cosine

In [ ]:
import ast
data["player_position"] = data["player_position"].apply(ast.literal_eval)


In [ ]:
def get_angle(shot):
  post1 = [120.0, 36.0] 
  post2 = [120.0,44.0] 
  shot_post1 = math.sqrt((120 - shot[0])**2 + (shot[1] - post1[1])**2)
  shot_post2 = math.sqrt((120 - shot[0])**2 + (shot[1] - post2[1])**2)
  cosa = (shot_post1**2 + shot_post2**2 - 64)/(2*shot_post1*shot_post2)

  return math.degrees(math.acos(round(cosa, 5)))

In [ ]:
data["dist_to_center"] = np.abs(data["y_pos"] - 40)

data["angle"] = data["player_position"].apply(get_angle)


In [ ]:
data["angle"]

0        13.958666
1        13.401453
2        14.613443
3        26.199788
4         6.434817
           ...    
22327     6.184577
22328    36.258197
22329    24.023297
22330    73.300629
22331    46.569830
Name: angle, Length: 22332, dtype: float64

#Obstructing Players Feature

In [ ]:

def point_in_triangle(shot, post1, post2, point):
  w1 =  (shot[0]*(post2[1] - shot[1]) + (point[1] - shot[1])*(post2[0] - shot[0]) - point[0]*(post2[1] - shot[1]))/((post1[1] - shot[1])*(post2[0] - shot[0]) - (post1[0] - shot[0])*(post2[1] - shot[0]))
  w2 = (point[1] - shot[1] - w1*(post1[1] - shot[1]))/(post2[1] - shot[1])
  return w1 >= 0 and w2 >= 0 and (w1 + w2) < 1

In [ ]:
post1 = [120.0, 36.0] 
post2 = [120.0,44.0]
obstr_col = []
for i in range(len(data)):
  obstructing_players = []
  shot = data["player_position"].iloc[0]
  for j in data["oppnent_info"].iloc[i]["location"]:
    obstructing_players.append(point_in_triangle(shot,post1, post2, j))
  obstr_col.append(sum(obstructing_players))

In [ ]:
data["obstructing_players"] = obstr_col
data


,Unnamed: 0,Unnamed: 0.1,statsbomb_xg,end_location,freeze_frame,player_position,first_time,aerial_won,deflected,open_goal,saved_off_target,saved_to_post,redirect,follows_dribble,kick_off,shot_outcome,shot_technique,shot_type,shot_body,x_pos,y_pos,Corner,Free Kick,Kick Off,Open Play,Penalty,Backheel,Diving Header,Half Volley,Lob,Normal,Overhead Kick,Volley,Blocked,Goal,Off T,Post,Saved,Saved Off Target,Saved to Post,Wayward,distance,teammate_info,oppnent_info,nearby_players,dist_to_center,angle,obstructing_players
0,0,0,0.021567,"[116.9, 41.6, 2.2]","[{'location': [89.6, 38.9], 'player': {'id': 6...","[99.2, 55.9]",0,0,0,0,0,0,0,0,0,Saved,Normal,Open Play,Right Foot,99.2,55.9,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,26.120490,position...,posi...,1,15.9,13.958666,0
1,1,1,0.018274,"[120.0, 44.7, 3.1]","[{'location': [117.2, 38.6], 'player': {'id': ...","[94.5, 25.1]",0,0,0,0,0,0,0,0,0,Off T,Normal,Open Play,Left Foot,94.5,25.1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,29.584624,position...,pos...,1,14.9,13.401453,0
2,2,2,0.016565,"[99.1, 31.3]","[{'location': [102.7, 32.4], 'player': {'id': ...","[95.3, 27.2]",0,0,0,0,0,0,0,0,0,Wayward,Normal,Open Play,Right Foot,95.3,27.2,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,27.865750,position...,pos...,0,12.8,14.613443,0
3,3,3,0.036084,"[120.0, 36.9, 0.3]","[{'location': [108.8, 43.1], 'player': {'id': ...","[106.0, 33.1]",1,0,0,0,0,0,0,0,0,Goal,Half Volley,Open Play,Right Foot,106.0,33.1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,15.652476,position...,posi...,0,6.9,26.199788,0
4,4,4,0.060245,"[120.0, 35.8, 3.8]","[{'location': [109.7, 36.0], 'player': {'id': ...","[108.1, 66.8]",1,0,0,0,0,0,0,0,0,Off T,Normal,Open Play,Left Foot,108.1,66.8,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,29.231832,posi...,posi...,0,26.8,6.434817,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22327,22679,22679,0.091634,"[120.0, 39.3, 1.2]","[{'location': [103.9, 61.2], 'player': {'id': ...","[93.6, 75.6]",0,0,0,0,0,0,0,0,0,Goal,Normal,Free Kick,Left Foot,93.6,75.6,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,44.240366,posi...,posi...,0,35.6,6.184577,0
22328,22680,22680,0.060538,"[120.0, 22.8, 0.8]","[{'location': [97.6, 45.2], 'player': {'id': 3...","[108.0, 41.7]",0,0,0,0,0,0,0,0,0,Off T,Normal,Open Play,Head,108.0,41.7,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,12.106197,position ...,position ...,0,1.7,36.258197,0
22329,22681,22681,0.154427,"[120.0, 43.4, 0.8]","[{'location': [108.0, 34.1], 'player': {'id': ...","[101.2, 40.0]",0,0,0,0,0,0,0,0,0,Goal,Normal,Free Kick,Left Foot,101.2,40.0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,18.800266,posi...,pos...,0,0.0,24.023297,0
22330,22682,22682,0.273795,"[118.8, 41.1, 2.0]","[{'location': [104.2, 30.8], 'player': {'id': ...","[114.8, 41.2]",0,1,0,0,0,0,0,0,0,Saved,Normal,Open Play,Head,114.8,41.2,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,5.315073,posi...,posi...,2,1.2,73.300629,0


In [ ]:
data = data.set_index("Unnamed: 0")


In [ ]:
len(data["angle"])

22332

In [ ]:
np.save("/content/drive/My Drive/obstructing.npy", np.array(data["obstructing_players"]))
np.save("/content/drive/My Drive/angles.npy", np.array(data["angle"]))

In [ ]:
data[data["angle"].isna()]

,Unnamed: 0.1,statsbomb_xg,end_location,freeze_frame,player_position,first_time,aerial_won,deflected,open_goal,saved_off_target,saved_to_post,redirect,follows_dribble,kick_off,shot_outcome,shot_technique,shot_type,shot_body,x_pos,y_pos,Corner,Free Kick,Kick Off,Open Play,Penalty,Backheel,Diving Header,Half Volley,Lob,Normal,Overhead Kick,Volley,Blocked,Goal,Off T,Post,Saved,Saved Off Target,Saved to Post,Wayward,distance,teammate_info,oppnent_info,nearby_players,dist_to_center,angle,obstructing_players
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [ ]:
data.to_csv("/content/drive/My Drive/shotoutcome_wthplayerinfo.csv")